# Imports

In [7]:
!pip pandas nltk sklearn language_tool_python gensim textstat joblib

ERROR: unknown command "pandas"



In [1]:
import pandas as pd
import nltk
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re
from language_tool_python import LanguageTool
from sklearn.feature_extraction.text import CountVectorizer
import textstat
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from gensim import models
from gensim import similarities
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import xgboost as xgb

In [12]:
# NLTK RESOURCES

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# Accessing dataset

In [12]:
# original_dataset = pd.read_csv('DATA_SET/final_test.csv')

# subset_dataset = original_dataset.sample(n=20, random_state=42)

# subset_dataset.to_csv('the123.csv', index=False)

In [21]:
df = pd.read_csv('the123.csv')
df.head(-1)

,text,label
0,I think that animals should not be used in sci...,1
1,Focus On The ROAS\n\nDo you think you should b...,0
2,Taking online or watching video conferencing c...,1
3,COMMUNITY_SERVICE\n\nYES.! I feel that all kid...,0
4,Driverless cars would be a Wood idea. It would...,0
5,Humans are like penguins. Just as penguins hud...,0
6,Transportation is a large necessity in most co...,0
7,On my experiences I think It would be impossib...,0
8,What would you do if you have a problem? Any p...,0
9,Many authors write about how we should do cert...,0


# Data preprocessing

In [22]:
# Handling missing values

print(df.isna().sum())

df = df.dropna()

text     0
label    0
dtype: int64


In [23]:
# Data Cleaning

def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    words = word_tokenize(text)

    cleaned_text = ' '.join(words)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

df['cleaned_text'] = df['text'].apply(clean_text)

In [24]:
# Space remover

def space_remover(text):
    words = word_tokenize(text)
    text = ' '.join(words)
    return re.sub(r'\s+', ' ', text).strip()

df['text'] = df['text'].apply(space_remover)

In [25]:
df['label'].value_counts()

label
0    15
1     5
Name: count, dtype: int64

In [18]:
df.head(20)

,text,label,cleaned_text
0,I think that animals should not be used in sci...,1,i think that animals should not be used in sci...
1,Focus On The ROAS Do you think you should be o...,0,focus on the roas do you think you should be o...
2,Taking online or watching video conferencing c...,1,taking online or watching video conferencing c...
3,COMMUNITY_SERVICE YES. ! I feel that all kids ...,0,communityservice yes i feel that all kids shou...
4,Driverless cars would be a Wood idea . It woul...,0,driverless cars would be a wood idea it would ...
5,Humans are like penguins . Just as penguins hu...,0,humans are like penguins just as penguins hudd...
6,Transportation is a large necessity in most co...,0,transportation is a large necessity in most co...
7,On my experiences I think It would be impossib...,0,on my experiences i think it would be impossib...
8,What would you do if you have a problem ? Any ...,0,what would you do if you have a problem any pe...
9,Many authors write about how we should do cert...,0,many authors write about how we should do cert...


# Feature engineering

In [17]:
# Basic NLP Features

df['char_count'] = df['cleaned_text'].apply(len) # char count

df['word_count'] = df['cleaned_text'].apply(lambda x: len(word_tokenize(x))) # word count

df['word_density'] = df['word_count'] / df['char_count'] # word density

# punctuation count
def punctuation_count(text):
    return sum(1 for char in text if char in string.punctuation)

df['punctuation_count'] = df['text'].apply(punctuation_count)

# Upper case count
def upper_case_count(text):
    return sum(1 for char in text if char.isupper())

df['upper_case_count'] = df['text'].apply(upper_case_count)

def title_word_count(text):
    return sum(1 for word in text.split() if word.istitle())

df['title_word_count'] = df['text'].apply(title_word_count)

# parts of speech
def parts_of_speech(text):
    pos_tags = pos_tag(word_tokenize(text))
    
    noun_count = sum(1 for tag in pos_tags if tag[1] in ['NN', 'NNS', 'NNP', 'NNPS'])
    adv_count = sum(1 for tag in pos_tags if tag[1] in ['RB', 'RBR', 'RBS'])
    verb_count = sum(1 for tag in pos_tags if tag[1] in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'])
    adj_count = sum(1 for tag in pos_tags if tag[1] in ['JJ', 'JJR', 'JJS'])
    pro_count = sum(1 for tag in pos_tags if tag[1] in ['PRP', 'PRP$', 'WP', 'WP$'])
    return pd.Series([noun_count, adv_count, verb_count, adj_count, pro_count], index=['noun_count','adv_count','verb_count','adj_count','pro_count'])

df[['noun_count','adv_count','verb_count','adj_count','pro_count']] = df['cleaned_text'].apply(lambda x: parts_of_speech(x))

In [67]:
print(df['cleaned_text'][3])

COMMUNITYSERVICE YES I feel that all kids should do Community service I feel this need because it takes up time and it also helps our Community Furthermore I think that the principal should make all students in the high and elementary schools do Community service Just think about it if every student in the schools did this we would never have to worry about tutors babysitters or even trash pickers and many other helpful things Also I feel that we all help put the trash down so why not help pick a little up every now and then I mean its not that hard The principal board of education or even the police should all help us to keep OUR environment Jean as well THINK ABOUT IT you do not want people to Some in GORDON or any other town and see a whole bunch of garbage thrown on the ground Also Im sure you would not want to go through the schools files and see a whole bunch of bad grades when you Would tutor the ones younger than you that need help That is about all I Jan says about the Communi

In [30]:
tool = LanguageTool('en-US')

In [31]:
from pandarallel import pandarallel

In [33]:
# Text error length - grammer mistakes

def error_length(text):
    matches = tool.check(text)
    return len(matches)

df['error_length'] = df['cleaned_text'].apply(error_length)

# pandarallel.initialize(progress_bar=True)

# def text_error_length(texts):
#     matches = tool.check(texts)
#     return [len(match) for match in matches]

# df['error_length'] = df['cleaned_text'].parallel_apply(text_error_length)

In [44]:
df.head()

,text,label
0,I think that animals should not be used in sci...,1
1,Focus On The ROAS\n\nDo you think you should b...,0
2,Taking online or watching video conferencing c...,1
3,COMMUNITY_SERVICE\n\nYES.! I feel that all kid...,0
4,Driverless cars would be a Wood idea. It would...,0


In [14]:
df = pd.read_csv('subset_dataset.csv')

In [56]:
df.head()

,text,label,cleaned_text,punctuation_count_1,punctuation_count_2
0,I think that animals should not be used in sci...,1,I think that animals should not be used in sci...,26,26
1,Focus On The ROAS\n\nDo you think you should b...,0,Focus On The ROAS Do you think you should be o...,44,44
2,Taking online or watching video conferencing c...,1,Taking online or watching video conferencing c...,17,17
3,COMMUNITY_SERVICE\n\nYES.! I feel that all kid...,0,COMMUNITY_SERVICE YES. ! I feel that all kids ...,39,39
4,Driverless cars would be a Wood idea. It would...,0,Driverless cars would be a Wood idea . It woul...,27,27


In [57]:
# Named Entity Recognition (NER)

def ner_count(text):
    words = word_tokenize(text)
    pos_tags = pos_tag(words)
    ner_tags = ne_chunk(pos_tags)
    ner_count = sum(1 for chunk in ner_tags if hasattr(chunk, 'label'))
    return ner_count

df['ner_count'] = df['text'].apply(ner_count)

In [45]:
df.head(-1)

,text,label
0,I think that animals should not be used in sci...,1
1,Focus On The ROAS\n\nDo you think you should b...,0
2,Taking online or watching video conferencing c...,1
3,COMMUNITY_SERVICE\n\nYES.! I feel that all kid...,0
4,Driverless cars would be a Wood idea. It would...,0
...,...,...
994,My argument about the 10pm curfew for teenager...,0
995,There are a variety of opinions on what the pr...,1
996,"Summer break, most students favorite time of t...",0
997,"In today's world, technology has become a Inte...",1


In [60]:
df['ner_count_1'].value_counts()

ner_count_1
0      1034
1       979
2       792
3       703
4       538
       ... 
104       1
67        1
74        1
65        1
69        1
Name: count, Length: 83, dtype: int64

In [24]:
# Readability score

df['flesch_kincaid_score'] = df['cleaned_text'].apply(lambda x: textstat.flesch_kincaid_grade(x))

df['flesch_score'] = df['cleaned_text'].apply(lambda x: textstat.flesch_reading_ease(x))

df['gunning_fog_score'] = df['cleaned_text'].apply(lambda x: textstat.gunning_fog(x))

df['coleman_liau_score'] = df['cleaned_text'].apply(lambda x: textstat.coleman_liau_index(x))

df['dale_chall_score'] = df['cleaned_text'].apply(lambda x: textstat.dale_chall_readability_score(x))

df['ari_score'] = df['cleaned_text'].apply(lambda x: textstat.automated_readability_index(x))

df['linsear_write_score'] = df['cleaned_text'].apply(lambda x: textstat.linsear_write_formula(x))

df['spache_score'] = df['cleaned_text'].apply(lambda x: textstat.spache_readability(x))

In [116]:
# Topic modeling - LDA

corpus = [text.split() for text in df['cleaned_text']]

# Create a dictionary
dictionary = corpora.Dictionary(corpus)

# Create a bag of words representation of the corpus
corpus_bow = [dictionary.doc2bow(text) for text in corpus]

# Train LDA model
num_topics = 20
lda_model = LdaModel(corpus_bow, num_topics=num_topics, id2word=dictionary, passes=15)

# Extract topic distribution for each document
topic_distribution = lda_model.get_document_topics(corpus_bow)

# Create columns for each topic score in the DataFrame
for topic in range(num_topics):
    df[f'topic_{topic + 1}_score'] = [next((t[1] for t in topic_dist if t[0] == topic), 0) for topic_dist in topic_distribution]

In [107]:
df['aan'].value_counts()

aan
0    996
1      2
2      1
3      1
Name: count, dtype: int64

In [117]:
df.head(-1)

,text,label,cleaned_text,topic_1_score,topic_2_score,topic_3_score,topic_4_score,topic_5_score,topic_6_score,topic_7_score,...,topic_11_score,topic_12_score,topic_13_score,topic_14_score,topic_15_score,topic_16_score,topic_17_score,topic_18_score,topic_19_score,topic_20_score
0,I think that animals should not be used in sci...,1,i think that animals should not be used in sci...,0.000000,0.000000,0.132977,0.000000,0.640100,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.192493,0.030645,0.0,0.000000,0.000000
1,Focus On The ROAS\n\nDo you think you should b...,0,focus on the roas do you think you should be o...,0.184806,0.000000,0.000000,0.701085,0.111935,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,Taking online or watching video conferencing c...,1,taking online or watching video conferencing c...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.025099,0.000000,0.0,0.000000,0.894534
3,COMMUNITY_SERVICE\n\nYES.! I feel that all kid...,0,communityservice yes i feel that all kids shou...,0.068814,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.014018,0.000000,0.798496,0.000000,0.0,0.000000,0.114917
4,Driverless cars would be a Wood idea. It would...,0,driverless cars would be a wood idea it would ...,0.000000,0.000000,0.000000,0.886622,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.091686,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,My argument about the 10pm curfew for teenager...,0,my argument about the pm curfew for teenagers ...,0.000000,0.029578,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.968079,0.000000,0.0,0.000000,0.000000
995,There are a variety of opinions on what the pr...,1,there are a variety of opinions on what the pr...,0.000000,0.000000,0.191261,0.000000,0.010954,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.120207,0.323758,0.000000,0.0,0.000000,0.303317
996,"Summer break, most students favorite time of t...",0,summer break most students favorite time of th...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
997,"In today's world, technology has become a Inte...",1,in todays world technology has become a integr...,0.000000,0.725967,0.271355,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [19]:
# Count Vectorizer

count_vect = df['cleaned_text'].tolist()

count_vectorizer = CountVectorizer()

count_matrix = count_vectorizer.fit_transform(count_vect)

count_df = pd.DataFrame(count_matrix.toarray(), columns=count_vectorizer.get_feature_names_out())

df = pd.concat([df, count_df], axis=1)


# count_vectorizer = CountVectorizer(max_features=1000, stop_words='english')

# # Transform text data to count matrix
# count_matrix = count_vectorizer.fit_transform(df['cleaned_text'])

# # Convert the count matrix to a DataFrame
# count_df = pd.DataFrame(count_matrix.toarray(), columns=count_vectorizer.get_feature_names_out())

# # Concatenate the count_df with the original DataFrame
# df = pd.concat([df, count_df], axis=1)


In [10]:
unique_words = count_vectorizer.get_feature_names_out()
print(len(unique_words))

1319


In [30]:
# df['error_length'].value_counts()
df.head(-1)

,text,label,cleaned_text,grin spoon,in the,it would,of the,on the,to be,to learn,...,your happy or,your life because,your location ans,your phone before,your phone is,your phone while,your problem studies,your pull over,your service while,your time to
0,I think that animals should not be used in sci...,1,i think that animals should not be used in sci...,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Focus On The ROAS Do you think you should be o...,0,focus on the roas do you think you should be o...,0,1,0,1,6,0,0,...,0,0,1,1,1,1,0,1,2,0
2,Taking online or watching video conferencing c...,1,taking online or watching video conferencing c...,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,COMMUNITY_SERVICE YES. ! I feel that all kids ...,0,communityservice yes i feel that all kids shou...,0,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Driverless cars would be a Wood idea . It woul...,0,driverless cars would be a wood idea it would ...,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Humans are like penguins . Just as penguins hu...,0,humans are like penguins just as penguins hudd...,0,2,0,2,0,3,0,...,0,1,0,0,0,0,0,0,0,0
6,Transportation is a large necessity in most co...,0,transportation is a large necessity in most co...,0,2,1,2,0,2,0,...,0,0,0,0,0,0,0,0,0,1
7,On my experiences I think It would be impossib...,0,on my experiences i think it would be impossib...,0,0,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,What would you do if you have a problem ? Any ...,0,what would you do if you have a problem any pe...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9,Many authors write about how we should do cert...,0,many authors write about how we should do cert...,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
df['zon'].value_counts()

zon
0    19
1     1
Name: count, dtype: int64

In [27]:
# Bigram Vectorizer

bigram_vect = df['cleaned_text'].tolist()

bigram_vectorizer = CountVectorizer(ngram_range=(2, 2), max_features=10)

bigram_matrix = bigram_vectorizer.fit_transform(bigram_vect)

bigram_df = pd.DataFrame(bigram_matrix.toarray(), columns=bigram_vectorizer.get_feature_names_out())

df = pd.concat([df, bigram_df], axis=1)

In [29]:
# Trigram Vectorizer

trigram_vect = df['cleaned_text'].tolist()

trigram_vectorizer = CountVectorizer(ngram_range=(3, 3), max_features=5000)

trigram_matrix = trigram_vectorizer.fit_transform(trigram_vect)

trigram_df = pd.DataFrame(trigram_matrix.toarray(), columns=trigram_vectorizer.get_feature_names_out())

df = pd.concat([df, trigram_df], axis=1)

In [31]:
# Bi-Trigram Vectorizer

bitri_vect = df['cleaned_text'].tolist()

bitri_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 3), max_features=5000)

bichar_matrix = bitri_vectorizer.fit_transform(bitri_vect)

bichar_df = pd.DataFrame(bichar_matrix.toarray(), columns=bitri_vectorizer.get_feature_names_out())

df = pd.concat([df, bichar_df], axis=1)


In [33]:
df.head(20)

# df.isna().sum()
# df.dropna(inplace=True)

,text,label,cleaned_text,grin spoon,in the,it would,of the,on the,to be,to learn,...,za,zat,ze,ze,zed,zes,zi,zin,zo,zon
0,I think that animals should not be used in sci...,1,i think that animals should not be used in sci...,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Focus On The ROAS Do you think you should be o...,0,focus on the roas do you think you should be o...,0,1,0,1,6,0,0,...,0,0,0,0,0,0,1,1,0,0
2,Taking online or watching video conferencing c...,1,taking online or watching video conferencing c...,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,COMMUNITY_SERVICE YES. ! I feel that all kids ...,0,communityservice yes i feel that all kids shou...,0,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Driverless cars would be a Wood idea . It woul...,0,driverless cars would be a wood idea it would ...,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Humans are like penguins . Just as penguins hu...,0,humans are like penguins just as penguins hudd...,0,2,0,2,0,3,0,...,0,0,2,0,2,0,0,0,0,0
6,Transportation is a large necessity in most co...,0,transportation is a large necessity in most co...,0,2,1,2,0,2,0,...,0,0,0,0,0,0,0,0,1,1
7,On my experiences I think It would be impossib...,0,on my experiences i think it would be impossib...,0,0,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,What would you do if you have a problem ? Any ...,0,what would you do if you have a problem any pe...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Many authors write about how we should do cert...,0,many authors write about how we should do cert...,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0


# Saving the dataset

In [4]:
df.to_csv('DATA_SET/pre_final_test.csv', index=False)

NameError: name 'df' is not defined

# Training the model

In [3]:
df = pd.read_csv('DATA_SET/results/basic_nlp.csv')

In [10]:
df.head(-1)

,text,label,cleaned_text,char_count,word_count,word_density,punctuation_count,upper_case_count,title_word_count,noun_count,adv_count,verb_count,adj_count,pro_count
0,We should keep the Electoral College for a num...,0,keep elector colleg number reason usual though...,1128,174,0.154255,43,56,56,106,5,22,29,0
1,More and more money is spent on building theat...,1,money spent build theater sport stadium make p...,386,66,0.170984,17,10,10,31,2,18,9,0
2,Limiting car usage can actually be effective b...,0,limit car usag actual effect belief limit car ...,943,165,0.174973,34,21,20,93,10,31,25,0
3,"Dear Mrs. Smith,\n\nI am writing to you today ...",1,dear mr smith write today gave opinion new pol...,1164,192,0.164948,47,32,32,86,7,35,41,0
4,"Dear Principal,\n\nAfter school or during scho...",0,dear princip school school activ like basketba...,1084,185,0.170664,78,51,35,80,13,35,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346971,Tie Face on Mars is really just a big misunder...,0,tie face mar realli big misunderstand strongli...,570,101,0.177193,16,29,24,43,4,20,23,1
346972,The whole purpose of democracy is to create a ...,0,whole purpos democraci creat govern reflect de...,1300,195,0.150000,46,38,32,113,5,29,30,0
346973,I firmly believe that governments worldwide sh...,1,firmli believ govern worldwid take immedi acti...,1519,224,0.147465,58,36,32,141,6,17,53,0
346974,I DFN't agree with this decision because a LFT...,0,dfnt agre decis lft ff kid may nft want tf apa...,988,203,0.205466,24,187,41,112,4,37,33,0


In [11]:
# want to know how many na is there in the dataset
df.isna().sum()


text                 0
label                0
cleaned_text         0
char_count           0
word_count           0
word_density         0
punctuation_count    0
upper_case_count     0
title_word_count     0
noun_count           0
adv_count            0
verb_count           0
adj_count            0
pro_count            0
dtype: int64

In [7]:
df.dropna(inplace=True)

In [8]:
# Support Vector Machine (SVM)

y = df['label']

X = df.drop(['text', 'label', 'cleaned_text'], axis=1)
# X = df.drop(['text', 'label'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVC()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.81
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.89      0.86     44511
           1       0.78      0.66      0.71     24885

    accuracy                           0.81     69396
   macro avg       0.80      0.78      0.78     69396
weighted avg       0.81      0.81      0.81     69396



In [12]:
# Random Forest Classifier

y = df['label']

# Features (excluding columns not used for training)
X = df.drop(['text', 'label', 'cleaned_text'], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.92
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94     44511
           1       0.93      0.84      0.88     24885

    accuracy                           0.92     69396
   macro avg       0.92      0.90      0.91     69396
weighted avg       0.92      0.92      0.92     69396



In [14]:
# XGBoost

y = df['label']

X = df.drop(['text','cleaned_text', 'label'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.87
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.93      0.90     44511
           1       0.85      0.76      0.80     24885

    accuracy                           0.87     69396
   macro avg       0.86      0.84      0.85     69396
weighted avg       0.87      0.87      0.87     69396



# Saving the model

In [9]:
joblib.dump(model, 'svm_model.joblib')

['svm_model.joblib']